In [0]:
# Read CSV file
df = spark.read .option("header", True) .option("inferSchema", True) .csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")


In [0]:

#Convert CSV to Delta format  
df.write .format("delta") .option("overwriteSchema", "true") .mode("overwrite").saveAsTable("default.ecommerce_events")


In [0]:
#Create Delta tables (SQL and PySpark)
spark.sql("""
CREATE TABLE events_delta
USING DELTA
AS SELECT * FROM ecommerce_events
""")


In [0]:
#Test schema enforcement
try:
    wrong_schema_df = spark.createDataFrame(
        [("x", "y", "z")],
        ["col1", "col2", "col3"]
    )
    
    wrong_schema_df.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable("default.ecommerce_events")

except Exception as e:
    print("Schema enforcement working:", e)


In [0]:
# Handle duplicate inserts
dedup_df = df.dropDuplicates(["event_time", "user_id", "product_id"])

dedup_df.write.format("delta").mode("overwrite").saveAsTable("default.ecommerce_events")